<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/02_nfl_predict_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

# 02. Feature Engineering

## Reading data

In [3]:
PATH = 'https://raw.githubusercontent.com/rafabandoni/nfl-predict/refs/heads/main/data/output/'

In [4]:
treated_df = pd.read_parquet(PATH + 'treated_df.parquet')
treated_df.head()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_scrm_plys_home_,defense_fumbles_ff_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_td_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_pdef_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_scoring_sfty_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_scrm_plys_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_int_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_receiving_lng_home_,offense_receiving_rec_fum_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_fum_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_fc_home_,special_teams_kickoff_returns_fum_home_,special_teams_kickoff_returns_fg_blk_home_,special_teams_kickoff_returns_xp_blk_home_,special_teams_kickoffs_ko_home_,special_teams_kickoffs_tb_home_,special_teams_kickoffs_ret_avg_home_,special_teams_kickoffs_osk_home_,special_teams_kickoffs_osk_rec_home_,special_teams_kickoffs_oob_home_,special_teams_kickoffs_td_home_,special_teams_punt_returns_avg_home_,special_teams_punt_returns_ret_home_,special_teams_punt_returns_pret_t_home_,special_teams_punt_returns_fc_home_,special_teams_punt_returns_fum_home_,special_teams_punt_returns_p_blk_home_,special_teams_punting_att_home_,special_teams_punting_cmp__perc_home_,special_teams_punting_yds/att_home_,special_teams_punting_int_home_,special_teams_punting_40+_home_,special_teams_punting_lng_home_,special_teams_punting_sck_home_,defense_downs_3rd_att_away_,defense_downs_3rd_md_away_,defense_downs_4th_att_away_,defense_downs_rec_1st_away_,defense_downs_rec_1st_perc_away_,defense_downs_rush_1st_away_,defense_downs_scrm_plys_away_,defense_fumbles_ff_away_,defense_fumbles_fr_td_away_,defense_fumbles_rec_fum_away_,defense_fumbles_rush_fum_away_,defense_interceptions_int_away_,defense_interceptions_int_td_away_,defense_interceptions_lng_away_,defense_passing_att_away_,defense_passing_cmp__perc_away_,defense_passing_yds/att_away_,defense_passing_td_away_,defense_passing_40+_away_,defense_passing_lng_away_,defense_passing_sck_away_,defense_receiving_pdef_away_,defense_rushing_20+_away_,defense_rushing_40+_away_,defense_rushing_lng_away_,defense_scoring_sfty_away_,defense_tackles_solo_away_,offense_downs_3rd_att_away_,offense_downs_3rd_md_away_,offense_downs_4th_att_away_,offense_downs_rec_1st_away_,offense_downs_rec_1st_perc_away_,offense_downs_rush_1st_away_,offense_downs_scrm_plys_away_,offense_passing_cmp__perc_away_,offense_passing_yds/att_away_,offense_passing_int_away_,offense_passing_40+_away_,offe

In [5]:
featured_df = treated_df.copy()

## Lag

* Lag is a feature with time information

### Last score lag playing home and away

In [6]:
# def get_lag_by_team_date(data, lag, home=True):
#   final_lagged_df = pd.DataFrame()
#   is_home_team = 'team_home' if home==True else 'team_away'
#   is_home_score = 'score_home' if home==True else 'score_away'
#   new_column_name = 'home_lag_' if home==True else 'away_lag_'

#   for team in data[is_home_team].unique():
#     lag_df = data[data[is_home_team] == team]
#     lag_df = lag_df[['schedule_date', is_home_team, is_home_score]]
#     lag_df = lag_df.sort_values([is_home_team, 'schedule_date'])
#     lag_df = lag_df.set_index([is_home_team, 'schedule_date']).shift(lag).reset_index()
#     final_lagged_df = pd.concat([final_lagged_df, lag_df], axis=0, ignore_index=True)
#   final_lagged_df.rename(columns={
#       is_home_score : f'{new_column_name}{lag}'
#   }, inplace=True)
#   return final_lagged_df

In [7]:
# df_home_lag_1 = get_lag_by_team_date(treated_df, 1)
# df_home_lag_2 = get_lag_by_team_date(treated_df, 2)
# df_home_lag_3 = get_lag_by_team_date(treated_df, 3)

# df_home_lag = df_home_lag_1.merge(df_home_lag_2,
#                                   on=['team_home', 'schedule_date'],
#                                   how='left')
# df_home_lag = df_home_lag.merge(df_home_lag_3,
#                                   on=['team_home', 'schedule_date'],
#                                   how='left')

In [8]:
# df_away_lag_1 = get_lag_by_team_date(treated_df, 1, home=False)
# df_away_lag_2 = get_lag_by_team_date(treated_df, 2, home=False)
# df_away_lag_3 = get_lag_by_team_date(treated_df, 3, home=False)

# df_away_lag = df_away_lag_1.merge(df_away_lag_2,
#                                   on=['team_away', 'schedule_date'],
#                                   how='left')
# df_away_lag = df_away_lag.merge(df_away_lag_3,
#                                   on=['team_away', 'schedule_date'],
#                                   how='left')

In [9]:
# featured_df = featured_df.merge(df_home_lag,
#                                 on=['team_home', 'schedule_date'],
#                                 how='left')
# featured_df = featured_df.merge(df_away_lag,
#                                 on=['team_away', 'schedule_date'],
#                                 how='left')

### Wins and loses of those teams historically

In [10]:
historic = {}

def calculate_historic(row):
    team_1, team_2 = sorted([row['team_home'], row['team_away']])
    key = (team_1, team_2)
    if key not in historic:
        historic[key] = {'victories_team_1': 0, 'victories_team_2': 0, 'matches': 0}
    historic[key]['matches'] += 1

    if row['home_winner']:
      if row['team_home'] == team_1:
        historic[key]['victories_team_1'] += 1
      else:
        historic[key]['victories_team_2'] += 1
    else:
      if row['team_home'] == team_1:
        historic[key]['victories_team_2'] += 1
      else:
        historic[key]['victories_team_1'] += 1

    if row['team_home'] == team_1:
       return historic[key]['victories_team_1'] / historic[key]['matches']
    else:
      return historic[key]['victories_team_2'] / historic[key]['matches']

featured_df['home_win_historic_confront'] = featured_df.apply(calculate_historic, axis=1)
featured_df['away_win_historic_confront'] = 1 - featured_df['home_win_historic_confront']

## Average result feature

Average points last 3 games

In [11]:
# def last_three_games_result(featured_df, home=True):
#     team = 'team_home' if home==True else 'team_away'
#     score = 'score_home' if home==True else 'score_away'
#     df = featured_df[['schedule_date', team, score]].sort_values([team, 'schedule_date'])
#     df[f'last_3_scores_{team}'] = df[score].rolling(window=3).mean().shift(1)
#     return df

# home_team_avg = last_three_games_result(featured_df)
# away_team_avg = last_three_games_result(featured_df, home=False)

# featured_df = featured_df.merge(home_team_avg[['schedule_date', 'team_home', 'last_3_scores_team_home']],
#                                 on=['schedule_date', 'team_home'],
#                                 how='left')
# featured_df = featured_df.merge(away_team_avg[['schedule_date', 'team_away', 'last_3_scores_team_away']],
#                                 on=['schedule_date', 'team_away'],
#                                 how='left')

Average points per game last season

In [12]:
slice_df = featured_df.copy()
slice_df['schedule_season'] = slice_df['schedule_season'] + 1

home_score_per_season = slice_df[['schedule_season',
                                     'team_home',
                                     'score_home']].groupby(['schedule_season',
                                                             'team_home'], as_index=False).mean()
away_score_per_season = slice_df[['schedule_season',
                                     'team_away',
                                     'score_away']].groupby(['schedule_season',
                                                             'team_away'], as_index=False).mean()

home_score_per_season.rename(columns={'score_home' : 'avg_score_home_last_season'}, inplace=True)
away_score_per_season.rename(columns={'score_away' : 'avg_score_away_last_season'}, inplace=True)

featured_df = featured_df.merge(home_score_per_season,
                                on=['schedule_season', 'team_home'],
                                how='left')
featured_df = featured_df.merge(away_score_per_season,
                                on=['schedule_season', 'team_away'],
                                how='left')

Total points last season

In [13]:
slice_df = featured_df.copy()
slice_df['schedule_season'] = slice_df['schedule_season'] + 1

home_score_per_season = slice_df[['schedule_season',
                                     'team_home',
                                     'score_home']].groupby(['schedule_season',
                                                             'team_home'], as_index=False).sum()
away_score_per_season = slice_df[['schedule_season',
                                     'team_away',
                                     'score_away']].groupby(['schedule_season',
                                                             'team_away'], as_index=False).sum()

home_score_per_season.rename(columns={'score_home' : 'total_score_home_last_season'}, inplace=True)
away_score_per_season.rename(columns={'score_away' : 'total_score_away_last_season'}, inplace=True)

featured_df = featured_df.merge(home_score_per_season,
                                on=['schedule_season', 'team_home'],
                                how='left')
featured_df = featured_df.merge(away_score_per_season,
                                on=['schedule_season', 'team_away'],
                                how='left')

Average in-game stats

In [14]:
in_game_stats_columns = list(featured_df.columns[9:-15])
in_game_stats_columns_home = []
in_game_stats_columns_away = []
for item in in_game_stats_columns:
  if 'home' in item:
    in_game_stats_columns_home.append(item)
  elif 'away' in item:
    in_game_stats_columns_away.append(item)
  else:
    raise ValueError(f'{item} - Not home nor away')

def create_in_game_stats_df(featured_df, home=True):
  to_join_columns = ['schedule_season', 'team_home' if home == True else 'team_away']
  to_join_columns.extend(in_game_stats_columns_home if home == True else in_game_stats_columns_away)

  in_game_stats = featured_df[to_join_columns]
  in_game_stats = in_game_stats.groupby(['schedule_season',
                                         'team_home' if home == True else 'team_away'],as_index=False).mean()
  in_game_stats['schedule_season'] = in_game_stats['schedule_season'] + 1
  return in_game_stats

in_game_stats_home = create_in_game_stats_df(featured_df)
in_game_stats_away = create_in_game_stats_df(featured_df, home=False)

In [15]:
mean_feature_df = featured_df.drop(in_game_stats_columns, axis=1)
mean_feature_df = mean_feature_df.merge(in_game_stats_home,
                                        on=['schedule_season', 'team_home'],
                                        how='left')
mean_feature_df = mean_feature_df.merge(in_game_stats_away,
                                        on=['schedule_season', 'team_away'],
                                        how='left')

Cleaning NaN data because of lags and aggregation.

In [16]:
mean_feature_df.dropna(inplace=True)

## Data Leakage avoidance

We might be aware of **data leakage**.

Examples of data leakage in machine learning models:
* Target leakage: When a model learns from data that contains information about the target variable
* Data preprocessing leakage: When information from the test set is used during data preprocessing
* Using the same dataset for training and testing: This is an example of overfitting, which can lead to overoptimistic performance estimates
* Using future data to make predictions about the past: This can happen in time series problems

In [17]:
final_featured_df = mean_feature_df.copy()
final_featured_df.drop([
    'schedule_date', # unnecessary feature
    'schedule_season', # unnecessary feature
    'team_home', # unnecessary feature
    'score_home', # avoiding data leakage for our model
    'score_away', # avoiding data leakage for our model
    'team_away', # unnecessary feature
    'total_score' # avoiding data leakage for our model
], axis=1, inplace=True)

final_featured_df.reset_index(drop=True, inplace=True)
final_featured_df

,schedule_playoff,stadium_neutral,home_winner,special_teams_punt_returns_p_blk_away_,special_teams_punting_att_away_,special_teams_punting_cmp__perc_away_,special_teams_punting_yds/att_away_,special_teams_punting_int_away_,special_teams_punting_40+_away_,special_teams_punting_lng_away_,special_teams_punting_sck_away_,home_win_historic_confront,away_win_historic_confront,avg_score_home_last_season,avg_score_away_last_season,total_score_home_last_season,total_score_away_last_season,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_scrm_plys_home_,defense_fumbles_ff_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_td_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_pdef_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_scoring_sfty_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_scrm_plys_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_int_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_receiving_lng_home_,offense_receiving_rec_fum_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_fum_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_fc_home_,special_teams_kickoff_returns_fum_home_,special_teams_kickoff_returns_fg_blk_home_,special_teams_kickoff_returns_xp_blk_home_,special_teams_kickoffs_ko_home_,special_teams_kickoffs_tb_home_,special_teams_kickoffs_ret_avg_home_,special_teams_kickoffs_osk_home_,special_teams_kickoffs_osk_rec_home_,special_teams_kickoffs_oob_home_,special_teams_kickoffs_td_home_,special_teams_punt_returns_avg_home_,special_teams_punt_returns_ret_home_,special_teams_punt_returns_pret_t_home_,special_teams_punt_returns_fc_home_,special_teams_punt_returns_fum_home_,special_teams_punt_returns_p_blk_home_,special_teams_punting_att_home_,special_teams_punting_cmp__perc_home_,special_teams_punting_yds/att_home_,special_teams_punting_int_home_,special_teams_punting_40+_home_,special_teams_punting_lng_home_,special_teams_punting_sck_home_,defense_downs_3rd_att_away_,defense_downs_3rd_md_away_,defense_downs_4th_att_away_,defense_downs_rec_1st_away_,defense_downs_rec_1st_perc_away_,defense_downs_rush_1st_away_,defense_downs_scrm_plys_away_,defense_fumbles_ff_away_,defense_fumbles_fr_td_away_,defense_fumbles_rec_fum_away_,defense_fumbles_rush_fum_away_,defense_interceptions_int_away_,defense_interceptions_int_td_away_,defense_interceptions_lng_away_,defense_passing_att_away_,defense_passing_cmp__perc_away_,defense_passing_yds/att_away_,defense_passing_td_away_,defense_passing_40+_away_,defense_passing_lng_away_,defense_passing_sck_away_,defense_receiving_pdef_away_,defense_rushing_20+_away_,defense_rushing_40+_away_,defense_rushing_lng_away_,defense_

## Saving our feature df

In [18]:
final_featured_df.to_parquet('featured_df.parquet')